# **PIIranha for PI redaction**

The following notebook contains the initial usage of the PIIranha model to evaluate whether or not it is a good performer for the project.

Step 1: imports

In [2]:
!pip install trafilatura
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from trafilatura.core import *
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.2/132.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.8/837.8 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 117.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.8/263.8 kB 25.3 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 4.9.4
    Uninstalling lxml-4.9.4:
      Successfully uninstalled lxml-4.9.4


Step 2: load model

In [3]:
# Load PIIranha model
tokenizer = AutoTokenizer.from_pretrained("iiiorg/piiranha-v1-detect-personal-information")
model = AutoModelForTokenClassification.from_pretrained("iiiorg/piiranha-v1-detect-personal-information")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/19.7k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

DebertaV2ForTokenClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(251000, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=Tr

Step 3: mount subset of GovDocs dataset

In [4]:
from google.colab import drive
drive.mount('/content/drive')
govdocs_dir ='/content/drive/My Drive/ECE570/govdocs_testingdata'

Mounted at /content/drive


Step 4: create a list of the html files from the dataset

In [5]:
# Function to identify and read HTML files from the dataset
def find_html_files(directory):
    html_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file has an HTML extension
            if file.endswith(".html") or file.endswith(".htm"):
                html_files.append(os.path.join(root, file))
    return html_files

# Find HTML files in the dataset
html_files = find_html_files(govdocs_dir)

Step 5: for each html file, extract content with Trafilatura and use the model to remove PII

In [6]:
# helping with RAM memory errors
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()

def mask_pii(text, aggregate_redaction=True):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Get the model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted labels
    predictions = torch.argmax(outputs.logits, dim=-1)

    # Convert token predictions to word predictions
    encoded_inputs = tokenizer.encode_plus(text, return_offsets_mapping=True, add_special_tokens=True)
    offset_mapping = encoded_inputs['offset_mapping']

    masked_text = list(text)
    is_redacting = False
    redaction_start = 0
    current_pii_type = ''
    count = 0

    for i, (start, end) in enumerate(offset_mapping):
        if start == end:  # Special token
            continue

        label = predictions[0][i].item()
        if label != model.config.label2id['O']:  # Non-O label
            pii_type = model.config.id2label[label]
            if not is_redacting:
                is_redacting = True
                redaction_start = start
                current_pii_type = pii_type
            elif not aggregate_redaction and pii_type != current_pii_type:
                # End current redaction and start a new one
                apply_redaction(masked_text, redaction_start, start, current_pii_type, aggregate_redaction)
                redaction_start = start
                current_pii_type = pii_type
                count += 1
        else:
            if is_redacting:
                apply_redaction(masked_text, redaction_start, end, current_pii_type, aggregate_redaction)
                is_redacting = False
                count += 1

    # Handle case where PII is at the end of the text
    if is_redacting:
        apply_redaction(masked_text, redaction_start, len(masked_text), current_pii_type, aggregate_redaction)
        count += 1

    return ''.join(masked_text), count

def apply_redaction(masked_text, start, end, pii_type, aggregate_redaction):
    for j in range(start, end):
        masked_text[j] = ''
    if aggregate_redaction:
        masked_text[start] = '[redacted]'
    else:
        masked_text[start] = f'[{pii_type}]'

# Process the HTML content with Trafilatura's extract() function
def process_html(html_content, chunk_size=512):
    text = extract(html_content, favor_recall=True)
    if text:
        redacted_text = ""
        instances = 0
        for i in range(0, len(text), chunk_size):
            chunk = text[i:i + chunk_size]
            redacted_chunk, chunk_instances = mask_pii(chunk, aggregate_redaction=False)
            redacted_text += redacted_chunk
            instances += chunk_instances
        return redacted_text, instances
    else:
        return None

# Function to read the content of an HTML file
def read_html_file(file_path):
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        return f.read()

# for each HTML file, extract content and detect and remove PII
if html_files:
    i = 0
    total_instances = 0
    chunk_size = 512
    for file in html_files:
        html_content = read_html_file(file)
        redacted_content, instances = process_html(html_content)
        total_instances += instances
        # if redacted_content:
        #     try:
        #         with open(f'./PIIranha_filtered_content/test{i}.txt', 'w', encoding='utf-8') as fp:
        #             fp.write(redacted_content)
        #         i += 1
        #     except Exception as e:
        #         print(f"Error writing to file: {e}")
        # else:
        #     print(f'Failed to extract text from {file}.')
    print("Total instances of PI detected in GovDocs subset: " + str(total_instances))

# test a specific instance: the ECE 404 homepage
ece404 = '/content/drive/My Drive/ECE570/ECE404.html'
with open(ece404, 'r', encoding='utf-8') as fp:
    ece404html_content = fp.read()
text = extract(ece404html_content, favor_recall=True) # favor_precision=True will cut out noise, favor_recall=True will keep more in
redacted_text, count = mask_pii(str(text), aggregate_redaction=False)
print("Total instances of PI detected in ECE404 homepage: " + str(count))
print(str(redacted_text))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Total instances of PI detected in GovDocs subset: 6574
Total instances of PI detected in ECE404 homepage: 13
Instructor:[I-GIVENNAME][I-SURNAME]
Professor, ECE
E-mail:[I-EMAIL]You must place the string 'ece404' in the subject line to get past your instructor's notorious spam filter)
Graduate TAs:[I-GIVENNAME][I-SURNAME]-
E-mail:[I-USERNAME][I-EMAIL][I-USERNAME]
Joseph Wang
-
E-mail:[I-USERNAME][I-EMAIL]
Adrien Dubois
-
E-mail:[I-USERNAME][I-EMAIL][I-USERNAME]
Lecture Location and Time
-
TuTh: 6:00 PM - 7:15 PM, PHYS 112
Course Description
-
Beyond question, computer and network security has emerged as one of
the most important subjects of study in modern times. Even the minutest
details of our lives now depend on our computers and networks working
with our trust that the information that is private to us will not fall
in the hands of those with ill intent. The two major components of
computer and network security are cryptography and what is known as
systems-oriented security. For a go

Step 6: Testing PIIranha further on a PII dataset

In [7]:
!pip install datasets
from datasets import load_dataset
ds = load_dataset("ai4privacy/pii-masking-400k")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

1en.jsonl:   0%|          | 0.00/84.8M [00:00<?, ?B/s]

de.jsonl:   0%|          | 0.00/82.7M [00:00<?, ?B/s]

es.jsonl:   0%|          | 0.00/42.5M [00:00<?, ?B/s]

...

fr.jsonl:   0%|          | 0.00/21.1M [00:00<?, ?B/s]

it.jsonl:   0%|          | 0.00/19.8M [00:00<?, ?B/s]

data/validation/nl.jsonl:   0%|          | 0.00/9.67M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/325517 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/81379 [00:00<?, ? examples/s]

Step 7: test using the validation data split

In [16]:
import re # for regular expressions

# select 1,000 rows from the validation set as no test set is available
data = ds['validation']
data = data[0:1000]
y_true = data['masked_text']
X = data['source_text']
y_pred = []
for x in X:
  y = mask_pii(x, aggregate_redaction=False)
  y_pred.append(y)

def normalize_text(text):
    text = text.replace(" ", "")
    text = text.replace("\n", "")
    return text.lower()

# finds each redaction and adds the redaction and the characters to its left and right into a tuple
# returns list of the tuples
def extract_redactions_with_context(text):
    redactions_with_context = []

    # Use regular expression to find all redactions and their positions
    for match in re.finditer(r'\[.*?\]', text):
        start = match.start()
        end = match.end()
        left_char = text[start - 1] if start > 0 else ''  # Get the left character
        right_char = text[end] if end < len(text) else ''  # Get the right character
        redactions_with_context.append((match.group(0), left_char, right_char))  # Store redaction and its context

    return redactions_with_context

# wrote own accuracy score function to compute accuracy of redactions based on what characters are to
# the left and right of each [redacted] segment
def compute_accuracy(y_true, y_pred):
    # normalize both true and predicted texts by removing all spaces, newlines, and converting to lowercase
    y_true_normalized = [normalize_text(str(t)) for t in y_true]
    y_pred_normalized = [normalize_text(str(p[0])) for p in y_pred]
    total_redactions = 0
    correct_matches = 0

    for true_text, pred_text in zip(y_true_normalized, y_pred_normalized):
      # find each redaction and the characters to its left and right, returns list of tuples
      true_redactions = extract_redactions_with_context(true_text)
      pred_redactions = extract_redactions_with_context(pred_text)

      # count total redactions in the true text
      total_redactions += len(pred_redactions)

      # check surrounding characters of predicted redactions against true redactions
      i = 0
      for pred_redaction, pred_left, pred_right in pred_redactions:
        if i < len(true_redactions) and true_redactions[i][1] == pred_left:
          correct_matches += 1
        if pred_right == '[': # the case when there are two back to back labels
            if i+1 < len(pred_redactions) and i < len(true_redactions) and pred_redactions[i+1][2] == true_redactions[i][2]:
              correct_matches += .5
        elif i < len(true_redactions) and true_redactions[i][2] == pred_right:
          correct_matches += .5
        i += 1

    # compute accuracy as the number of correct matches over the total redactions
    accuracy = correct_matches / total_redactions if total_redactions > 0 else 0
    return accuracy

# Calculate accuracy
accuracy = compute_accuracy(y_true, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 89.49%
